In [7]:
import requests
import csv
import time
import random

# Cấu hình header
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://tiki.vn/sac-dung-phong/c1821',
}

def crawl_powerbanks_from_category(max_pages=3):
    all_products = []
    category_name = "Sạc dự phòng"
    category_id = 1821  # ID chính xác cho danh mục Sạc dự phòng

    for page in range(1, max_pages + 1):
        print(f"📊 Đang thu thập trang {page}/{max_pages}...")
        
        url = "https://tiki.vn/api/v2/products"
        params = {
            "limit": 17,
            "category": category_id,
            "page": page,
            "sort": "newest"
        }

        try:
            response = requests.get(
                url,
                headers=headers,
                params=params,
                timeout=10
            )

            if response.status_code == 200:
                data = response.json()
                products = data.get("data", [])
                
                if not products:
                    print(f"⏩ Không có sản phẩm ở trang {page}, dừng thu thập")
                    break
                
                for p in products:
                    # Chỉ lấy các trường theo yêu cầu
                    product = {
                        "category": category_name,
                        "product_name": p.get('name', ''),
                        "price": p.get('price', 0),
                        "image": p.get('thumbnail_url', ''),
                        "source": f"https://tiki.vn/{p.get('url_path', '')}"
                    }
                    all_products.append(product)
                
                print(f"✅ Đã thêm {len(products)} sản phẩm từ trang {page}")
            else:
                print(f"⚠️ Lỗi {response.status_code} ở trang {page}")
                print(f"   Thông báo: {response.text[:100]}...")
                break  # Dừng nếu gặp lỗi

        except Exception as e:
            print(f"❌ Lỗi kết nối: {str(e)}")
            break

        # Delay ngẫu nhiên
        time.sleep(random.uniform(1.0, 2.5))

    return all_products

# Thu thập sản phẩm
print("🛒 Bắt đầu thu thập sạc dự phòng từ Tiki...")
products = crawl_powerbanks_from_category(max_pages=3)

# Ghi file CSV chỉ với 5 trường
if products:
    with open("tiki_sac_du_phong.csv", "w", newline="", encoding="utf-8-sig") as f:
        fieldnames = ["category", "product_name", "price", "image", "source"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(products)
    
    print(f"💾 Đã lưu {len(products)} sản phẩm vào tiki_sac_du_phong.csv")
    
    # Xem thử sản phẩm đầu tiên
    first_product = products[0]
    print("\nThông tin mẫu:")
    print(f"Danh mục: {first_product['category']}")
    print(f"Tên SP: {first_product['product_name']}")
    print(f"Giá: {first_product['price']:,} VND")
    print(f"Ảnh: {first_product['image'][:50]}...")
    print(f"Link: {first_product['source']}")
else:
    print("⛔ Không thu thập được sản phẩm nào")

🛒 Bắt đầu thu thập sạc dự phòng từ Tiki...
📊 Đang thu thập trang 1/3...
✅ Đã thêm 17 sản phẩm từ trang 1
📊 Đang thu thập trang 2/3...
✅ Đã thêm 17 sản phẩm từ trang 2
📊 Đang thu thập trang 3/3...
✅ Đã thêm 17 sản phẩm từ trang 3
💾 Đã lưu 51 sản phẩm vào tiki_sac_du_phong.csv

Thông tin mẫu:
Danh mục: Sạc dự phòng
Tên SP: Pin sạc dự phòng 66W sạc siêu nhanh SHOUHUZHE SH-171 dung lượng 10000mAh sạc nhanh hai chiều, tiêu chuẩn 3C quốc gia, hàng chính hãng
Giá: 237,650 VND
Ảnh: https://salt.tikicdn.com/cache/280x280/ts/product/...
Link: https://tiki.vn/pin-sac-du-phong-66w-sac-sieu-nhanh-shouhuzhe-sh-171-dung-luong-10000mah-sac-nhanh-hai-chieu-tieu-chuan-3c-quoc-gia-hang-chinh-hang-p278350975.html?spid=278351008
